In [1]:
pip install google-adk -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

- import required elements

In [9]:
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner

#MCP Related Imports
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

- Retry Configurations setting

In [4]:
retry_congig = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503,504]
)

- Trying to connect to a MCP Server i.e., Everything Server

-- We need to create McpToolSet for configuring

In [14]:
mcp_image_server = McpToolset(
                    connection_params= StdioConnectionParams(
                        server_params= StdioServerParameters(
                            command="npx",
                                      args=["-y", "@modelcontextprotocol/server-everything",],
                                      tool_filter = ["getTinyImage"],
                        ),
                        timeout = 30,
                        
                    ) )

- Creating an agent and adding this tool from mcp

In [15]:
image_agent = LlmAgent(
    name = "ImageAgent",
    model = Gemini(
        model_name="gemini-1.5-pro",
        retry_options=retry_congig,
    ),
    instruction= "Create an image using the MCP Tool",
    tools = [mcp_image_server]
)

In [17]:
runner = InMemoryRunner(agent=image_agent)

In [23]:
response = runner.run_debug("Provide a sample tiny image", verbose=True)

- Connecting to Kaggle MCP Server

In [27]:
kaggle_server = McpToolset(
    connection_params = StdioConnectionParams(
        server_params= StdioServerParameters(
            command="npx",
            args= [ "-y", "mcp-remote", "https://kaggle.com/mcp" ],
        ),
        timeout=30,
    )
)

In [28]:
kaggle_data_agent = LlmAgent(
    name = "KaggleDataAgent",
    model = Gemini(
        model_name="gemini-1.5-pro",
        retry_options=retry_congig,
    ),
    instruction= "Use the Kaggle MCP Tool to get dataset information",
    tools = [kaggle_server]
)

In [29]:
KaggleRunner = InMemoryRunner(agent=kaggle_data_agent)

In [30]:
response = runner.run_debug("Give me a small dataset from kaggle mcp server")

In [31]:
response

<coroutine object Runner.run_debug at 0x0000016ECC508040>